# Functions for grabbing data from WRC API

This package contains a range of functions for grabbing and parsing live timing results data from the WRC website via a simple JSON API that is used to generate the official WRC live timing results web pages.


TO DO - consider a scraper class with a requests session embedded in it.

In [12]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize

In [13]:
# Cache results in text notebook
import requests_cache
requests_cache.install_cache('wrc_cache',
                             backend='sqlite',
                             expire_after=300)

In [14]:
s = requests.Session()
s.get('https://www.wrc.com')

<Response [200]>

In [4]:
# TO DO 
# There is also an: activeSeasonId":19
# IS there something we can get there?

In [15]:
#Is this URL constant or picked up relative to each rally?
URL='https://www.wrc.com/ajax.php?contelPageId=176146'

In [154]:
def _getresponse(_url, args):
    """Simple function to get response from a post request."""
    r = s.post(_url, data=json.dumps(args))
    return r

def _get_and_handle_response(_url, args, func, nargs=1, raw=False):
    r =  _getresponse(_url,args) 

    if raw or not callable(func):
        return r.text
    
    if not r.text or r.text=='null':
        return tuple([None for i in range(nargs)])
    
    return func(r)

In [130]:
ACTIVE_RALLY_URL = 'https://www.wrc.com/ajax.php?contelPageId=171091'

In [155]:
def _parseActiveRally(r):
    """Parse active rally response."""
    event = json_normalize(r.json()).drop(columns='eventDays')
    days =  json_normalize(r.json(), 'eventDays').drop(columns='spottChannel.assets')
    channels = json_normalize(r.json(), ['eventDays', 'spottChannel','assets'])
    return (event, days, channels)

def getActiveRallyBase(_url=None, raw=False, func=_parseActiveRally):
    """Get active rally details."""
    
    if not _url:
        _url = ACTIVE_RALLY_URL  
    args= {"command":"getActiveRally","context":None}
    
    return _get_and_handle_response(_url, args, func, raw)


In [156]:
event, days, channels = getActiveRallyBase() #also works with passing URL
display(event.head())
display(days.head())
display(channels.head())
display(event.columns)

,id,name,externalIdRally,externalIdEvent,timezone,active,countdown,jwrc,images.format16x9.320x180,images.format16x9.160x90,...,winner.nation.alpha3,winner.nation.ioc,winner.birthDate,winner.birthPlace,winner.debutDate,winner.debutPlace,winner.website,winner.driverImageFormats,winner.externalId,winner.page
0,100,Rallye Monte Carlo,153,124,1,True,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,BEL,BEL,1988-06-16,Belgium,2009-01-12,Rally de Portugal,https://www.thierryneuville.com/,"[{'id': 6, 'title': 'Format 16:9', 'imageForma...",762,None


,id,eventDay,spottChannel.id,spottChannel.displayName
0,331,2020-01-22,2,WRC Rallye Monte Carlo
1,334,2020-01-23,2,WRC Rallye Monte Carlo
2,341,2020-01-24,2,WRC Rallye Monte Carlo
3,344,2020-01-25,2,WRC Rallye Monte Carlo
4,355,2020-01-26,2,WRC Rallye Monte Carlo


,id,start,startUnix,end,endUnix,duration,alternative.title,alternative.description,alternative.image.480x270,alternative.image.thumbnail,...,content.image.800x450,content.image.thumbnail,content.image.original,content.dateTime.date,content.dateTime.timezone_type,content.dateTime.timezone,content.payment.id,content.payment.name,content.status.id,content.status.name
0,227,2020-01-22T15:00:00+00:00,1579705200,2020-01-22T15:40:00+00:00,1579707600,1548,Preview Magazine,None,https://ott.wrc.com/image/480/270/5e1e0da6bd29...,https://ott.wrc.com/image/thumbnail/5e1e0da6bd...,...,https://ott.wrc.com/image/800/450/5e21bfd8e014...,https://ott.wrc.com/image/thumbnail/5e21bfd8e0...,https://ott.wrc.com/image/original/5e21bfd8e01...,2020-01-17 14:30:00.000000,1,+00:00,3,Pay,3,Live
1,64,2020-01-23T18:00:00+00:00,1579802400,2020-01-23T18:30:00+00:00,1579804200,0,Good Evening Rally Fans - Service Gap,None,https://ott.wrc.com/image/480/270/5e2618150992...,https://ott.wrc.com/image/thumbnail/5e26181509...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
2,65,2020-01-23T18:30:00+00:00,1579804200,2020-01-23T19:15:00+00:00,1579806900,0,eSPORTS (TV Live),None,https://ott.wrc.com/image/480/270/5e1e100fce9e...,https://ott.wrc.com/image/thumbnail/5e1e100fce...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
3,239,2020-01-23T19:15:00+00:00,1579806900,2020-01-23T19:30:00+00:00,1579807800,0,Break,None,https://ott.wrc.com/image/480/270/5e25b39e1be1...,https://ott.wrc.com/image/thumbnail/5e25b39e1b...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
4,68,2020-01-23T19:30:00+00:00,1579807800,2020-01-23T20:30:00+00:00,1579811400,0,SS1 Malijai - Puimichel (TV LIVE),None,https://ott.wrc.com/image/480/270/5e1dfc31d968...,https://ott.wrc.com/image/thumbnail/5e1dfc31d9...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live


Index(['id', 'name', 'externalIdRally', 'externalIdEvent', 'timezone',
       'active', 'countdown', 'jwrc', 'images.format16x9.320x180',
       'images.format16x9.160x90', 'images.format16x9.path', 'season.id',
       'season.year', 'season.externalId', 'season.active', 'rally.id',
       'rally.name', 'rally.nation.id', 'rally.nation.name',
       'rally.nation.isoNumCode', 'rally.nation.alpha2', 'rally.nation.alpha3',
       'rally.nation.ioc', 'status.id', 'status.name', 'pageInfo.id',
       'pageInfo.title', 'pageInfo.feTitle', 'pageInfo.url', 'pageResult.id',
       'pageResult.title', 'pageResult.feTitle', 'pageResult.url', 'winner.id',
       'winner.firstName', 'winner.middleName', 'winner.lastName',
       'winner.nation.id', 'winner.nation.name', 'winner.nation.isoNumCode',
       'winner.nation.alpha2', 'winner.nation.alpha3', 'winner.nation.ioc',
       'winner.birthDate', 'winner.birthPlace', 'winner.debutDate',
       'winner.debutPlace', 'winner.website', 'winner.drive

In [7]:
#Raw https://webappsdata.wrc.com/srv API?
#Need to create separate package to query that API
#Season info
#_url = 'https://webappsdata.wrc.com/srv/wrc/json/api/wrcsrv/byType?t=%22Season%22&maxdepth=1' 
#r = s.get(_url)
#json_normalize(r.json())

In [106]:
CURRENT_SEASON_URL = 'https://www.wrc.com/ajax.php?contelPageId=181782'

In [137]:
def _parseCurrentSeasonEvents(r):
    """Parse current season events response."""
    current_season_events = json_normalize(r.json(), ['rallyEvents', 'items'], meta='seasonYear').drop(columns='eventDays')
    eventdays = json_normalize(r.json(), ['rallyEvents', 'items', 'eventDays']).drop(columns='spottChannel.assets')
    eventchannel = json_normalize(r.json(), ['rallyEvents', 'items', 'eventDays', 'spottChannel','assets'])
    return (current_season_events, eventdays, eventchannel)

def getCurrentSeasonEvents(raw=False, func=_parseCurrentSeasonEvents):
    """Get events for current season"""
    _url=CURRENT_SEASON_URL
    #There seems to be a second UTL giving same data?
    #_url='https://www.wrc.com/ajax.php?contelPageId=183400'
    args = {"command":"getActiveSeason","context":None}

    return _get_and_handle_response(_url, args, func, nargs=3, raw=raw)
        


In [139]:
current_season_events, eventdays, eventchannel = getCurrentSeasonEvents()
display(current_season_events.head())
display(eventdays.head())
display(eventchannel.head())
display(current_season_events.columns)

,id,name,externalIdRally,externalIdEvent,timezone,active,countdown,jwrc,images.format16x9.320x180,images.format16x9.160x90,...,winner.birthDate,winner.birthPlace,winner.debutDate,winner.debutPlace,winner.website,winner.driverImageFormats,winner.externalId,winner.page,winner,seasonYear
0,100,Rallye Monte Carlo,153,124,1,True,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,1988-06-16,Belgium,2009-01-12,Rally de Portugal,https://www.thierryneuville.com/,"[{'id': 6, 'title': 'Format 16:9', 'imageForma...",762.0,NaN,NaN,2020
1,102,Rally Sweden,154,125,2,False,False,True,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
2,107,Rally Guanajuato Mexico,155,126,-6,False,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
3,114,Rally Argentina,156,127,-3,False,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
4,116,Rally de Portugal,157,128,1,False,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020


,id,eventDay,spottChannel.id,spottChannel.displayName
0,331,2020-01-22,2,WRC Rallye Monte Carlo
1,334,2020-01-23,2,WRC Rallye Monte Carlo
2,341,2020-01-24,2,WRC Rallye Monte Carlo
3,344,2020-01-25,2,WRC Rallye Monte Carlo
4,355,2020-01-26,2,WRC Rallye Monte Carlo


,id,start,startUnix,end,endUnix,duration,alternative.title,alternative.description,alternative.image.480x270,alternative.image.thumbnail,...,content.image.800x450,content.image.thumbnail,content.image.original,content.dateTime.date,content.dateTime.timezone_type,content.dateTime.timezone,content.payment.id,content.payment.name,content.status.id,content.status.name
0,227,2020-01-22T15:00:00+00:00,1579705200,2020-01-22T15:40:00+00:00,1579707600,1548,Preview Magazine,None,https://ott.wrc.com/image/480/270/5e1e0da6bd29...,https://ott.wrc.com/image/thumbnail/5e1e0da6bd...,...,https://ott.wrc.com/image/800/450/5e21bfd8e014...,https://ott.wrc.com/image/thumbnail/5e21bfd8e0...,https://ott.wrc.com/image/original/5e21bfd8e01...,2020-01-17 14:30:00.000000,1,+00:00,3,Pay,3,Live
1,64,2020-01-23T18:00:00+00:00,1579802400,2020-01-23T18:30:00+00:00,1579804200,0,Good Evening Rally Fans - Service Gap,None,https://ott.wrc.com/image/480/270/5e2618150992...,https://ott.wrc.com/image/thumbnail/5e26181509...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
2,65,2020-01-23T18:30:00+00:00,1579804200,2020-01-23T19:15:00+00:00,1579806900,0,eSPORTS (TV Live),None,https://ott.wrc.com/image/480/270/5e1e100fce9e...,https://ott.wrc.com/image/thumbnail/5e1e100fce...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
3,239,2020-01-23T19:15:00+00:00,1579806900,2020-01-23T19:30:00+00:00,1579807800,0,Break,None,https://ott.wrc.com/image/480/270/5e25b39e1be1...,https://ott.wrc.com/image/thumbnail/5e25b39e1b...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
4,68,2020-01-23T19:30:00+00:00,1579807800,2020-01-23T20:30:00+00:00,1579811400,0,SS1 Malijai - Puimichel (TV LIVE),None,https://ott.wrc.com/image/480/270/5e1dfc31d968...,https://ott.wrc.com/image/thumbnail/5e1dfc31d9...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live


Index(['id', 'name', 'externalIdRally', 'externalIdEvent', 'timezone',
       'active', 'countdown', 'jwrc', 'images.format16x9.320x180',
       'images.format16x9.160x90', 'images.format16x9.path', 'season.id',
       'season.year', 'season.externalId', 'season.active', 'rally.id',
       'rally.name', 'rally.nation.id', 'rally.nation.name',
       'rally.nation.isoNumCode', 'rally.nation.alpha2', 'rally.nation.alpha3',
       'rally.nation.ioc', 'status.id', 'status.name', 'pageInfo.id',
       'pageInfo.title', 'pageInfo.feTitle', 'pageInfo.url', 'pageResult.id',
       'pageResult.title', 'pageResult.feTitle', 'pageResult.url', 'winner.id',
       'winner.firstName', 'winner.middleName', 'winner.lastName',
       'winner.nation.id', 'winner.nation.name', 'winner.nation.isoNumCode',
       'winner.nation.alpha2', 'winner.nation.alpha3', 'winner.nation.ioc',
       'winner.birthDate', 'winner.birthPlace', 'winner.debutDate',
       'winner.debutPlace', 'winner.website', 'winner.drive

## getItinerary

In [ ]:
def getActiveRally():
    """Get active rally details."""
    event, days, channels = _getActiveRally(URL)
    return (event, days, channels)

In [ ]:
event, days, channels = getActiveRally()
display(event)
display(days)
display(channels.head())

In [142]:
#This seems to work with sdbRallyId=None, returning active rally?

def _parseItinerary(r):
    """Parse itninerary response."""
    itinerary = json_normalize(r.json()).drop(columns='itineraryLegs')
    legs = json_normalize(r.json(),'itineraryLegs' )
    if not legs.empty:
        legs = legs.drop(columns='itinerarySections')
        sections = json_normalize(r.json(),['itineraryLegs', 'itinerarySections'] ).drop(columns=['controls','stages'])
        controls = json_normalize(r.json(),['itineraryLegs', 'itinerarySections', 'controls' ] )
        stages = json_normalize(r.json(),['itineraryLegs', 'itinerarySections', 'stages' ] )
    else:
        legs = sections = controls = stages = None
    return (itinerary, legs, sections, controls, stages)

def getItinerary(sdbRallyId=None, raw=False, func=_parseItinerary):
    """Get itinerary details for specified rally."""
    args = {"command":"getItinerary",
            "context":{"sdbRallyId":sdbRallyId}}
    
    return _get_and_handle_response(URL, args, func, nargs=5, raw=raw)


In [146]:
sdbRallyId = 100
itinerary, legs, sections, controls, stages = getItinerary(sdbRallyId)
display(itinerary.head())
display(legs.head())
display(sections.head())
display(controls.head())
display(stages.head())

,itineraryId,eventId,name,priority
0,240,124,Itinerary,1


,itineraryLegId,itineraryId,startListId,name,legDate,order,status
0,273,240,451,Thursday 23rd January,2020-01-23,1,Completed
1,272,240,452,Friday 24th January,2020-01-24,2,Completed
2,275,240,454,Saturday 25th January,2020-01-25,3,Completed
3,274,240,456,Sunday 26th January,2020-01-25,4,Completed


,itinerarySectionId,itineraryLegId,order,name
0,637,273,1,Section 1
1,638,272,2,Section 2
2,639,272,3,Section 3
3,640,275,4,Section 4
4,641,275,5,Section 5


,controlId,eventId,stageId,type,code,location,timingPrecision,distance,targetDuration,targetDurationMs,firstCarDueDateTime,firstCarDueDateTimeLocal,status,controlPenalties,roundingPolicy,locked
0,6539,124,NaN,TimeControl,TC0,Monaco,Minute,0.00,None,NaN,2020-01-23T16:00:00,2020-01-23T17:00:00+01:00,Completed,All,NoRounding,True
1,6543,124,NaN,TimeControl,TC0A,Tyre Fitting Zone IN,Minute,166.33,02:45:00,9900000.0,2020-01-23T18:45:00,2020-01-23T19:45:00+01:00,Completed,All,NoRounding,True
2,6541,124,NaN,TimeControl,TC0B,Tyre Fitting Zone OUT,Minute,0.35,00:15:00,900000.0,2020-01-23T19:00:00,2020-01-23T20:00:00+01:00,Completed,All,NoRounding,True
3,6593,124,1528.0,TimeControl,TC1,Malijai,Minute,17.08,00:35:00,2100000.0,2020-01-23T19:35:00,2020-01-23T20:35:00+01:00,Completed,All,NoRounding,True
4,6592,124,1528.0,StageStart,SS1,Malijai - Puimichel (Live TV),Minute,17.47,00:03:00,180000.0,2020-01-23T19:38:00,2020-01-23T20:38:00+01:00,Interrupted,None,RoundToClosestMinute,True


,stageId,eventId,number,name,distance,status,stageType,timingPrecision,locked,code
0,1528,124,1,Malijai - Puimichel (Live TV),17.47,Interrupted,SpecialStage,Tenth,True,SS1
1,1538,124,2,Bayons - Bréziers,25.49,Completed,SpecialStage,Tenth,True,SS2
2,1533,124,3,Curbans - Venterol 1,20.02,Completed,SpecialStage,Tenth,True,SS3
3,1534,124,4,Saint-Clément - Freissinières 1,20.68,Completed,SpecialStage,Tenth,True,SS4
4,1535,124,5,Avançon - Notre-Dame-du-Laus 1,20.59,Completed,SpecialStage,Tenth,True,SS5


In [149]:
def _parseStartlist(r):
    """Parse raw startlist response."""
    startList = json_normalize(r.json()).drop(columns='startListItems')
    startListItems = json_normalize(r.json(), 'startListItems')
    
    return (startList,startListItems)

def getStartlist(startListId, raw=False, func=_parseStartlist):
    """Get a startlist given startlist ID."""
    args={'command': 'getStartlist',
          'context': {'activeItineraryLeg': { 'startListId': startListId} }}

    return _get_and_handle_response(URL, args, func, nargs=2, raw=raw)

In [150]:
startListId = 451
startList,startListItems = getStartlist(startListId)
display(startList.head())
display(startListItems.head())

,startListId,eventId,publishedStatus,name
0,451,124,Published,Thursday


,startListItemId,startListId,entryId,startDateTime,startDateTimeLocal,order
0,20891,451,20745,2020-01-23T17:37:00Z,2020-01-23T18:37:00+01:00,64
1,20892,451,20743,2020-01-23T17:36:00Z,2020-01-23T18:36:00+01:00,63
2,20893,451,20722,2020-01-23T17:35:00Z,2020-01-23T18:35:00+01:00,62
3,20894,451,20741,2020-01-23T17:34:00Z,2020-01-23T18:34:00+01:00,61
4,20895,451,20740,2020-01-23T17:33:00Z,2020-01-23T18:33:00+01:00,60


In [ ]:
def getCars(sdbRallyId):
    """Get cars for a specified rally."""
    args = {"command":"getCars","context":{"sdbRallyId":100}}
    r = s.post(URL, data=json.dumps(args))
    if not r.text or r.text=='null':
        cars = classes = None
        return (cars, classes)
    
    cars = json_normalize(r.json()).drop(columns='eventClasses')
    classes = json_normalize(r.json(), 'eventClasses')
    return (cars, classes)

In [ ]:
cars, classes = getCars(sdbRallyId)
display(cars.head())
display(classes.head())
cars.head().columns

In [ ]:
def getRally(sdbRallyId):
    """Get rally details for specified rally."""
    args = {"command":"getRally","context":{"sdbRallyId":sdbRallyId}}
    r = s.post(URL, data=json.dumps(args))
    rally = json_normalize(r.json()).drop(columns=['eligibilities','groups'])
    eligibilities = json_normalize(r.json(),'eligibilities')
    groups = json_normalize(r.json(),'groups')
    return (rally, eligibilities, groups)

In [ ]:
rally, eligibilities, groups = getRally(sdbRallyId)
display(rally.head())
display(eligibilities.head())
display(groups.head())

In [ ]:
def getOverall(sdbRallyId, stageId):
    """Get overall standings for specificed rally and stage."""
    args = {"command":"getOverall","context":{"sdbRallyId":sdbRallyId,
                                              "activeStage":{"stageId":stageId}}}
    r = s.post(URL, data=json.dumps(args))
    overall = json_normalize(r.json())
    return overall

In [ ]:
stageId = 1528
overall = getOverall(sdbRallyId, stageId)
overall.head()

In [ ]:
def getSplitTimes(sdbRallyId,stageId):
    """Get split times for specified rally and stage."""
    args = {"command":"getSplitTimes",
            "context":{"sdbRallyId":sdbRallyId, "activeStage":{"stageId":stageId}}}
    r = s.post(URL, data=json.dumps(args))
    splitPoints = json_normalize(r.json(),'splitPoints')
    entrySplitPointTimes = json_normalize(r.json(), 'entrySplitPointTimes').drop(columns='splitPointTimes')
    splitPointTimes = json_normalize(r.json(), ['entrySplitPointTimes','splitPointTimes'])
    return (splitPoints, entrySplitPointTimes, splitPointTimes)

In [ ]:
splitPoints, entrySplitPointTimes, splitPointTimes = getSplitTimes(sdbRallyId,stageId)
display(splitPoints.head())
display(entrySplitPointTimes.head())
display(splitPointTimes.head())

In [ ]:
def getStageTimes(sdbRallyId,stageId):
    """Get stage times for specified rally and stage"""
    args = {"command":"getStageTimes",
            "context":{"sdbRallyId":sdbRallyId,
                       "activeStage":{"stageId":stageId}}}
    r = s.post(URL, data=json.dumps(args))
    stagetimes = json_normalize(r.json())
    return stagetimes

In [ ]:
stagetimes = getStageTimes(sdbRallyId,stageId)
stagetimes.head()

In [ ]:
def getStagewinners(sdbRallyId):
    """Get stage winners for specified rally."""
    args = {"command":"getStagewinners",
            "context":{"sdbRallyId":sdbRallyId}}
    r = s.post(URL, data=json.dumps(args))
    stagewinners = json_normalize(r.json())
    return stagewinners

In [ ]:
stagewinners = getStagewinners(sdbRallyId)
stagewinners.head()

Should we return empty dataframes with appropriate columns, or `None`?

An advantage of returning an empty dataframe with labelled columns is that we can also use the column value list as a test of a returned column.

We need to be consistent so we can have a common, consistent way of dealing with empty responses. This means things like `is None` or `pd.DataFrame().empty` both have to be handled.

In [ ]:
COLS_PENALTIES=['penaltyId','controlId','entryId','penaltyDurationMs','penaltyDuration','reason']
               
def getPenalties(sdbRallyId):
    """Get penalties for specified rally."""
    args = {"command":"getPenalties",
            "context":{"sdbRallyId":sdbRallyId}}
    r = s.post(URL, data=json.dumps(args))
    
    if not r.text:
        return pd.DataFrame(columns=COLS_PENALTIES)
    
    penalties = json_normalize(r.json())
    return penalties

In [ ]:
penalties = getPenalties(sdbRallyId)
penalties.head()

In [ ]:
COLS_RETIREMENT = ['retirementId','controlId','entryId','reason','retirementDateTime','retirementDateTimeLocal','status']

def getRetirements(sdbRallyId):
    """Get retirements for specified rally."""
    args = {"command":"getRetirements",
            "context":{"sdbRallyId":sdbRallyId}}
    r = s.post(URL, data=json.dumps(args))
    
    if not r.text:
        return pd.DataFrame(columns=COLS_RETIREMENT)
    
    retirements = json_normalize(r.json())
    return retirements

In [ ]:
retirements = getRetirements(sdbRallyId)
retirements.head()

In [ ]:
SEASON_URL = 'https://www.wrc.com/ajax.php?contelPageId=186641'

In [ ]:
#How can we look these up?
SEASON_CATEGORIES = {'WRC':"35", "WRC2":"46", "WRC3":"49","JWRC":"58"}

In [ ]:
def getSeasonCategory(seasonCategory=SEASON_CATEGORIES['WRC']): 
    """Get championships in season category."""
    args = {"command":"getSeasonCategory",
            "context":{"seasonCategory":seasonCategory}}
    r = s.post(SEASON_URL, data=json.dumps(args))
    if not r.text:
        return None
    return json_normalize(r.json())

In [ ]:
getSeasonCategory()

In [ ]:
SC_COLS = ['id','externalIdDriver','externalIdCoDriver','externalIdManufacturer']

def getChampionshipCodes():
    """Create dataframe of external championship IDs."""
    champs=pd.DataFrame()

    for sc in SEASON_CATEGORIES:
        seasonCategory = SEASON_CATEGORIES[sc]

        champs = champs.append(getSeasonCategory(seasonCategory)[SC_COLS])

    champs.set_index('id', inplace=True)
    champs.rename(columns={'externalIdDriver':'drivers',
                           'externalIdCoDriver':'codrivers',
                           'externalIdManufacturer':'manufacturers'},
                 inplace=True)
    return champs


In [ ]:
getChampionshipCodes()

In [ ]:
def _getChampionshipId(category='WRC', typ='drivers'):
    """Look up external ids for championship by category and championship."""
    champs=getChampionshipCodes()
    championship_activeExternalId = champs.to_dict(orient='index')[int(SEASON_CATEGORIES[category])]
    activeExternalId = championship_activeExternalId[typ]
    return activeExternalId

def getChampionship(category='WRC',typ='drivers',
                    season_external_id=None, ):
    """Get Championship details for specified category and championship.
       If nor season ID is provided, use the external seasonid from the active rally. """
    
    season_external_id = SeasonBase(season_external_id, autoseed=True).season_external_id
    args = {"command":"getChampionship",
            "context":{"season":{"externalId":season_external_id},
                       "activeExternalId":_getChampionshipId(category,typ)}}
    
    r = s.post(SEASON_URL, data=json.dumps(args))

    if not r.text:
        championship = championshipRounds = championshipEntries = None
        return (championship, championshipRounds, championshipEntries)
    
    championship = json_normalize(r.json()).drop(columns=['championshipRounds','championshipEntries'])
    championshipRounds = json_normalize(r.json(), 'championshipRounds' )
    championshipEntries = json_normalize(r.json(), 'championshipEntries')
    return (championship, championshipRounds, championshipEntries)

In [ ]:
(championship, championshipRounds, championshipEntries) = getChampionship('JWRC', 'drivers')
display(championship)
display(championshipRounds.head())
display(championshipEntries.head())

In [ ]:
getChampionshipCodes().to_dict(orient='index')#[int(SEASON_CATEGORIES['JWRC'])]

In [ ]:
def getChampionshipStandings(category='WRC',typ='drivers',
                             season_external_id=None, ):
    """Get championship standings."""
    season_external_id = SeasonBase(season_external_id, autoseed=True).season_external_id
    args = {"command":"getChampionshipStandings",
            "context":{
                       "season":{"externalId":season_external_id,
                                 },
                       "activeExternalId":_getChampionshipId(category,typ)}}
    r = s.post(SEASON_URL, data=json.dumps(args))
    
    if not r.text:
        championship_standings = round_results = None
        return (championship_standings, round_results)
    championship_standings = json_normalize(r.json(),'entryResults', meta='championshipId').drop(columns='roundResults')
    round_results = json_normalize(r.json(),['entryResults', 'roundResults'])

    return (championship_standings, round_results)

In [ ]:
championship_standings, round_results = getChampionshipStandings()
display(championship_standings.head())
display(round_results.head())